# Chapter 15: Wallet Integration and User Authentication

---

Wallets are the gateway for users to interact with blockchain applications. They manage private keys, sign transactions, and provide a secure interface for DApps. In this chapter, we'll explore the different types of wallets, how to integrate them into your DApp, and how to implement secure user authentication using Sign-In with Ethereum (SIWE). By the end, you'll be able to connect your DApp to MetaMask, WalletConnect, and implement passwordless authentication.

---

## 15.1 Understanding Crypto Wallets

A **crypto wallet** is a software or hardware tool that stores private keys and allows users to interact with blockchain networks. It doesn't store coins or tokens—those live on the blockchain. Instead, the wallet stores the keys that prove ownership and enable transactions.

### 15.1.1 Hot Wallets vs. Cold Wallets

Wallets are categorized by how they store private keys—online or offline.

```
┌─────────────────────────────────────────────────────────────────┐
│                    HOT WALLETS                                   │
│  • Connected to the internet                                     │
│  • Convenient for frequent transactions                         │
│  • Higher risk of hacking                                       │
│  • Examples: MetaMask, Trust Wallet, Coinbase Wallet            │
└─────────────────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────────────────┐
│                    COLD WALLETS                                  │
│  • Offline storage (not connected to internet)                  │
│  • Secure for long-term holdings                                │
│  • Less convenient for frequent use                             │
│  • Examples: Hardware wallets (Ledger, Trezor), Paper wallets   │
└─────────────────────────────────────────────────────────────────┘
```

**Comparison:**

| Feature | Hot Wallet | Cold Wallet |
|---------|------------|-------------|
| Connectivity | Always online | Mostly offline |
| Security | Lower (exposed to online threats) | Higher (air-gapped) |
| Convenience | High (quick transactions) | Low (need to connect/approve) |
| Use Case | Daily transactions, DApps | Long-term storage, large holdings |
| Cost | Usually free | Hardware costs money |

### 15.1.2 Custodial vs. Non-Custodial

This distinction refers to who controls the private keys.

```
Custodial Wallet:
┌─────────────────────────────────────────────────────┐
│  Exchange / Service                                 │
│  ┌─────────────────────────────────────────────┐   │
│  │ Your private keys are held by the service   │   │
│  │ You log in with username/password           │   │
│  │ Service can freeze or lose your funds       │   │
│  └─────────────────────────────────────────────┘   │
│  Examples: Coinbase, Binance (exchange wallets)    │
└─────────────────────────────────────────────────────┘

Non-Custodial Wallet:
┌─────────────────────────────────────────────────────┐
│  Your Device                                        │
│  ┌─────────────────────────────────────────────┐   │
│  │ You control your private keys               │   │
│  │ No one can move funds without your approval │   │
│  │ You are your own bank                       │   │
│  └─────────────────────────────────────────────┘   │
│  Examples: MetaMask, Ledger, Trust Wallet          │
└─────────────────────────────────────────────────────┘
```

For DApps, we typically integrate **non-custodial** wallets because users expect to retain control of their keys. The DApp never sees the private key; it only requests signatures via the wallet.

### 15.1.3 Hardware Wallets

Hardware wallets are physical devices that store private keys offline. They are the most secure option for storing significant amounts of cryptocurrency.

**Popular hardware wallets:**
- **Ledger Nano X / S**: Supports many blockchains, Bluetooth on X model.
- **Trezor Model T**: Open-source, touchscreen.
- **KeepKey**: Basic but secure.

**How they work with DApps:**
Hardware wallets can be connected via USB or Bluetooth, and they interact with browsers through bridge software (Ledger Live, Trezor Bridge). When a DApp requests a transaction, the hardware wallet prompts the user to physically confirm on the device.

**Integration with DApps:**
```javascript
// Example using ethers.js with Ledger via Web3Provider
// This works if Ledger is connected and unlocked
const provider = new ethers.providers.Web3Provider(window.ethereum);
// The wallet appears as a normal Ethereum account
```

---

## 15.2 MetaMask Integration

MetaMask is the most widely used browser extension wallet. It injects a global `ethereum` object into web pages, which DApps can use to interact with the user's accounts.

### 15.2.1 Installing and Configuring MetaMask

**Installation:**
1. Visit [metamask.io](https://metamask.io) and download the extension for your browser.
2. Follow the setup wizard to create a new wallet or import an existing one.
3. Securely store the seed phrase (12 or 24 words).

**Configuration for developers:**
- Enable "Show test networks" in settings to see Sepolia, Goerli, etc.
- Add custom networks if needed (e.g., local Hardhat node at `http://localhost:8545`).

### 15.2.2 Detecting MetaMask in DApps

Your DApp should check if MetaMask (or any injected provider) is available.

```javascript
// Check if ethereum object exists
if (typeof window.ethereum !== 'undefined') {
  console.log('MetaMask is installed!');
} else {
  console.log('Please install MetaMask');
  // Show a message or redirect to installation page
}
```

To handle multiple injected wallets (e.g., Coinbase Wallet, Brave wallet), you can use `window.ethereum` detection, but for better compatibility, use libraries like `@web3-react` or `web3modal`.

### 15.2.3 Connecting Wallet to DApp

Connecting involves requesting account access from the user. This triggers a MetaMask popup asking for permission.

**Using ethers.js:**
```javascript
async function connectWallet() {
  if (typeof window.ethereum === 'undefined') {
    alert('Please install MetaMask');
    return;
  }

  try {
    // Request account access
    await window.ethereum.request({ method: 'eth_requestAccounts' });

    // Create ethers provider
    const provider = new ethers.providers.Web3Provider(window.ethereum);
    const signer = provider.getSigner();
    const address = await signer.getAddress();
    const network = await provider.getNetwork();

    console.log('Connected:', address);
    console.log('Network:', network.name);

    // Store in state / context
    return { provider, signer, address, network };
  } catch (error) {
    console.error('User rejected connection', error);
  }
}
```

**Common pitfalls:**
- User may reject the connection request.
- Always handle the rejection gracefully.
- After connection, listen for account or network changes.

### 15.2.4 Switching Networks

Users may need to switch to a specific network (e.g., Sepolia for testing). You can request MetaMask to switch or add a network.

**Switch network:**
```javascript
async function switchNetwork(chainId) {
  // chainId should be in hex, e.g., '0xaa36a7' for Sepolia
  try {
    await window.ethereum.request({
      method: 'wallet_switchEthereumChain',
      params: [{ chainId }],
    });
  } catch (switchError) {
    // This error code indicates that the chain has not been added to MetaMask
    if (switchError.code === 4902) {
      // Need to add the network first
      // See next section
    } else {
      console.error('Switch failed', switchError);
    }
  }
}
```

### 15.2.5 Adding Custom Networks

If the network isn't configured in MetaMask, you can add it programmatically.

```javascript
async function addNetwork() {
  try {
    await window.ethereum.request({
      method: 'wallet_addEthereumChain',
      params: [{
        chainId: '0xaa36a7', // Sepolia
        chainName: 'Sepolia Testnet',
        nativeCurrency: {
          name: 'Sepolia ETH',
          symbol: 'ETH',
          decimals: 18
        },
        rpcUrls: ['https://sepolia.infura.io/v3/YOUR_PROJECT_ID'],
        blockExplorerUrls: ['https://sepolia.etherscan.io']
      }],
    });
  } catch (error) {
    console.error('Add network failed', error);
  }
}
```

---

## 15.3 WalletConnect and Mobile Wallets

WalletConnect is an open protocol that connects DApps to mobile wallets via a QR code or deep linking. It's essential for mobile users who can't use browser extensions.

### 15.3.1 WalletConnect Protocol

WalletConnect works by establishing a bridge between the DApp and the wallet using a relay server. The connection is initiated by scanning a QR code or clicking a deep link.

**How it works:**
1. DApp generates a connection URI (with a session topic and encryption keys).
2. Displays it as a QR code or link.
3. Mobile wallet scans the QR code, establishes a secure WebSocket connection through a relay.
4. All future messages are encrypted and sent via the relay.

**Benefits:**
- Works with many mobile wallets (MetaMask Mobile, Trust Wallet, Rainbow, etc.).
- No browser extension needed.
- Secure end-to-end encryption.

### 15.3.2 Integration Steps

To integrate WalletConnect, you can use the `@walletconnect/web3-provider` package or a higher-level library like `web3modal`.

**Using Web3Modal (recommended):**
Web3Modal provides a unified interface for multiple wallet providers, including MetaMask and WalletConnect.

```bash
npm install web3modal @walletconnect/web3-provider
```

**Example setup:**
```javascript
import Web3Modal from 'web3modal';
import WalletConnectProvider from '@walletconnect/web3-provider';
import { ethers } from 'ethers';

const providerOptions = {
  walletconnect: {
    package: WalletConnectProvider,
    options: {
      rpc: {
        1: 'https://mainnet.infura.io/v3/YOUR_PROJECT_ID',
        5: 'https://goerli.infura.io/v3/YOUR_PROJECT_ID',
        // Add other networks
      }
    }
  }
};

const web3Modal = new Web3Modal({
  network: 'mainnet',
  cacheProvider: true,
  providerOptions
});

async function connect() {
  const provider = await web3Modal.connect();
  const ethersProvider = new ethers.providers.Web3Provider(provider);
  const signer = ethersProvider.getSigner();
  const address = await signer.getAddress();
  console.log('Connected via WalletConnect:', address);
}
```

Web3Modal will display a popup with available wallets; when the user selects WalletConnect, it shows a QR code for mobile wallets to scan.

---

## 15.4 SIWE (Sign-In with Ethereum)

Sign-In with Ethereum is an authentication mechanism that allows users to sign in to web applications using their Ethereum wallet, without needing a username and password. It's secure, decentralized, and gives users control over their identity.

### 15.4.1 Authentication Without Passwords

Traditional authentication relies on servers storing password hashes and managing sessions. With SIWE, the server issues a challenge (a nonce and message), the user signs it with their private key, and the server verifies the signature.

**Benefits:**
- No passwords to leak or forget.
- User's identity is their wallet address.
- Portable across applications.
- Can include additional data (like session duration, domain, etc.).

**How it works:**
```
┌─────────┐                              ┌─────────┐
│ Client  │                              │ Server  │
└────┬────┘                              └────┬────┘
     │                                         │
     │ 1. Request nonce                        │
     │────────────────────────────────────────>│
     │                                         │
     │ 2. Send nonce and message               │
     │<────────────────────────────────────────│
     │                                         │
     │ 3. Sign message (wallet)                │
     │    (no server involved)                 │
     │                                         │
     │ 4. Send signature                       │
     │────────────────────────────────────────>│
     │                                         │
     │ 5. Verify signature                     │
     │    Recover address, check nonce, domain │
     │    Create session                        │
     │<────────────────────────────────────────│
     │                                         │
     │ 6. Access granted                        │
```

### 15.4.2 Message Signing

The message format is standardized by [EIP-4361](https://eips.ethereum.org/EIPS/eip-4361). It includes:

- Domain (the origin of the DApp)
- Address (the user's wallet address)
- Statement (optional, what the user is agreeing to)
- URI (the DApp's URL)
- Version
- Chain ID
- Nonce (to prevent replay attacks)
- Issued At (timestamp)
- Expiration Time (optional)
- Resources (optional)

**Example message:**
```
example.com wants you to sign in with your Ethereum account:
0xAb5801a7D398351b8bE11C439e05C5B3259aeC9B

I accept the ExampleOrg Terms of Service: https://example.com/tos

URI: https://example.com
Version: 1
Chain ID: 1
Nonce: 328917
Issued At: 2023-01-01T00:00:00.000Z
Expiration Time: 2023-01-01T01:00:00.000Z
Resources:
- ipfs://Qme7ss3ARVgxv6rXqVPiikMJ8u2NLgmgszg13pYrDKEoiu
```

**Signing in the client:**
```javascript
async function signInWithEthereum() {
  const provider = new ethers.providers.Web3Provider(window.ethereum);
  const signer = provider.getSigner();
  const address = await signer.getAddress();

  // 1. Request nonce from server
  const response = await fetch('/api/nonce');
  const { nonce } = await response.json();

  // 2. Construct message according to EIP-4361
  const message = `${window.location.host} wants you to sign in with your Ethereum account:
${address}

I accept the Terms of Service.

URI: ${window.location.origin}
Version: 1
Chain ID: 1
Nonce: ${nonce}
Issued At: ${new Date().toISOString()}`;

  // 3. Sign message
  const signature = await signer.signMessage(message);

  // 4. Send signature to server for verification
  const verifyResponse = await fetch('/api/verify', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({ message, signature })
  });

  if (verifyResponse.ok) {
    console.log('Authenticated!');
    // Server returns a session token or sets a cookie
  }
}
```

### 15.4.3 Implementation Guide

Let's implement a simple SIWE backend using Node.js/Express.

**Server-side (Node.js/Express):**
```bash
npm install express siwe ethers cors
```

**Generate nonce endpoint:**
```javascript
const express = require('express');
const { SiweMessage } = require('siwe');
const cors = require('cors');
const crypto = require('crypto');

const app = express();
app.use(cors());
app.use(express.json());

// In-memory store for nonces (use Redis in production)
const nonces = new Set();

app.get('/api/nonce', (req, res) => {
  const nonce = crypto.randomBytes(16).toString('hex');
  nonces.add(nonce);
  res.json({ nonce });
});
```

**Verify signature endpoint:**
```javascript
app.post('/api/verify', async (req, res) => {
  const { message, signature } = req.body;

  try {
    // Parse the message
    const siweMessage = new SiweMessage(message);

    // Verify the signature and that the message hasn't been tampered with
    const fields = await siweMessage.verify({
      signature,
      nonce: siweMessage.nonce,
      domain: req.headers.origin // or use siweMessage.domain
    });

    // Check nonce was issued and not reused
    if (!nonces.has(siweMessage.nonce)) {
      return res.status(400).json({ error: 'Invalid nonce' });
    }
    nonces.delete(siweMessage.nonce); // prevent replay

    // At this point, authentication is successful
    // Create a session (JWT or session cookie)
    const sessionToken = crypto.randomBytes(32).toString('hex');
    // Store session with user's address (fields.data.address)

    res.json({ success: true, sessionToken });
  } catch (error) {
    console.error(error);
    res.status(400).json({ error: 'Invalid signature' });
  }
});
```

**Frontend usage:**
After verification, the server can return a JWT or set a secure cookie. Subsequent requests include this token to authenticate the user.

**Security considerations:**
- Always use HTTPS in production.
- Nonces should expire (e.g., 5 minutes).
- Verify domain matches to prevent phishing.
- Use proper session management.
- Consider using libraries like `siwe` (as above) which handle parsing and verification.

**Full frontend hook example:**
```javascript
import { useState } from 'react';
import { ethers } from 'ethers';

export function useSIWE() {
  const [authenticated, setAuthenticated] = useState(false);
  const [address, setAddress] = useState(null);

  const signIn = async () => {
    try {
      const provider = new ethers.providers.Web3Provider(window.ethereum);
      await provider.send('eth_requestAccounts', []);
      const signer = provider.getSigner();
      const userAddress = await signer.getAddress();
      setAddress(userAddress);

      // Get nonce
      const nonceRes = await fetch('/api/nonce');
      const { nonce } = await nonceRes.json();

      // Create message
      const message = `${window.location.host} wants you to sign in:
${userAddress}

Sign in to the app.

URI: ${window.location.origin}
Version: 1
Chain ID: 1
Nonce: ${nonce}
Issued At: ${new Date().toISOString()}`;

      const signature = await signer.signMessage(message);

      // Verify
      const verifyRes = await fetch('/api/verify', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ message, signature })
      });

      if (verifyRes.ok) {
        setAuthenticated(true);
        const { sessionToken } = await verifyRes.json();
        // Store token (e.g., in localStorage or cookie)
        localStorage.setItem('session', sessionToken);
      } else {
        throw new Error('Verification failed');
      }
    } catch (error) {
      console.error('Sign-in failed', error);
    }
  };

  const signOut = () => {
    setAuthenticated(false);
    setAddress(null);
    localStorage.removeItem('session');
  };

  return { authenticated, address, signIn, signOut };
}
```

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 15 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  • Wallets are essential for DApp interaction, storing keys     │
│  • Types: Hot vs Cold, Custodial vs Non-Custodial               │
│                                                                 │
│  MetaMask Integration:                                          │
│    - Detect window.ethereum                                     │
│    - Request accounts with eth_requestAccounts                  │
│    - Switch/add networks                                        │
│                                                                 │
│  WalletConnect:                                                 │
│    - Protocol for mobile wallets                                │
│    - Use Web3Modal for unified experience                       │
│                                                                 │
│  Sign-In with Ethereum (SIWE):                                  │
│    - Passwordless authentication                                │
│    - EIP-4361 standardized message format                      │
│    - Server verifies signature and nonce                        │
│    - Prevents replay attacks                                    │
│                                                                 │
│  SECURITY TIPS:                                                 │
│    • Always verify domain and chain ID                          │
│    • Use short-lived nonces                                     │
│    • Never trust signatures without verification                │
│    • Use HTTPS in production                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 16 – Layer 2 Scaling Solutions. We'll explore how to scale blockchain applications using rollups, sidechains, and state channels to achieve lower fees and higher throughput.